# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**: Luis Daniel Arellano Núñez

**Professor**: Pablo Camarillo Ramirez

# Dataset description

This dataset contains information about a github based social netowkr with nodes that are developers who have starred at least 10 repositories and edges are mutual follower relationships between them

You can find this dataset in this url:
https://www.kaggle.com/datasets/kausthubkannan/github-social-network/data

You only need to download the 2 csv and save them in the correspondig path in Data

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://8c99a6c586f5:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-632e73b1-a49c-490f-a418-cf2cfacc26a3;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [12]:
# Build schema
# Import your module
from Daniel_Arellano.sql_im import SparkUtils
from pyspark.sql.functions import regexp_replace, split, col

users_schema = SparkUtils.generate_schema([
    ("id", "int"),
    ("name", "string"),
    ("ml_target", "int")
])

df_users = spark.read \
    .option("header", "true") \
    .schema(users_schema) \
    .csv("/opt/spark/work-dir/data/lab6/nodes")

df_users.show(5, truncate=False)

edges_schema = SparkUtils.generate_schema([
    ("id_1", "int"),
    ("id_2", "int")
])

df_edges = spark.read \
    .option("header", "true") \
    .schema(edges_schema) \
    .csv("/opt/spark/work-dir/data/lab6/edges")
df_edges.show(5, truncate=False)


+---+------------+---------+
|id |name        |ml_target|
+---+------------+---------+
|0  |Eiryyy      |0        |
|1  |shawflying  |0        |
|2  |JpMCarrilho |1        |
|3  |SuhwanCha   |0        |
|4  |sunilangadi2|1        |
+---+------------+---------+
only showing top 5 rows
+----+-----+
|id_1|id_2 |
+----+-----+
|0   |23977|
|1   |34526|
|1   |2370 |
|1   |14683|
|1   |29982|
+----+-----+
only showing top 5 rows


# Transformations

In [13]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
from pyspark.sql.functions import col

# Artists Nodes
user_nodes = df_users.select(
    col("id").alias("id"),
    col("name"),
    col("ml_target")
).dropDuplicates(["id"])
user_nodes.show(5, truncate=False)

# --- EDGES ---

# Collaborations: relationships between User -> User
collab_edges = df_edges.select(
    col("id_1").alias("src"),   # source node (User 1)
    col("id_2").alias("dst"),   # destination node (User 2)
)
collab_edges.show(5, truncate=False)


+---+------------+---------+
|id |name        |ml_target|
+---+------------+---------+
|0  |Eiryyy      |0        |
|1  |shawflying  |0        |
|2  |JpMCarrilho |1        |
|3  |SuhwanCha   |0        |
|4  |sunilangadi2|1        |
+---+------------+---------+
only showing top 5 rows
+---+-----+
|src|dst  |
+---+-----+
|0  |23977|
|1  |34526|
|1  |2370 |
|1  |14683|
|1  |29982|
+---+-----+
only showing top 5 rows


# Writing Data in Neo4j

In [7]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"

user_nodes.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":User") \
  .option("node.keys", "id") \
  .save()

print(f"{user_nodes.count()} users wrote in Neo4j")

37700 users wrote in Neo4j


In [14]:
collab_edges.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "FOLLOWS") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":User") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:id") \
  .option("relationship.target.labels", ":User") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:id") \
  .option("batch.size", 500) \
  .save()

print(f"{collab_edges.count()} follows edges wrote in Neo4j")

4999 follows edges wrote in Neo4j


# Read and Query Graphs with PySpark

In [17]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 

cypher_df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            MATCH (u:User)-[r:FOLLOWS]->(u2:User)
            WHERE coalesce(u.ml_target, 0) > 0
            WITH u, u2, u.ml_target AS ml_target
            LIMIT 20
            RETURN u.name AS origin_user, u2.name AS dest_user, ml_target
            """) \
    .load()

cypher_df.show(5, truncate=False)

+------------+-------------+---------+
|origin_user |dest_user    |ml_target|
+------------+-------------+---------+
|danalex97   |benjaminaaron|1        |
|danalex97   |panpan2      |1        |
|sunilangadi2|mshahriarinia|1        |
|sunilangadi2|KnightBaron  |1        |
|Daichou     |penk         |1        |
+------------+-------------+---------+
only showing top 5 rows


In [18]:
sc.stop()